In [3]:
import os
import glob
import imageio
import random, shutil
import torch
import torch.nn as nn
from tqdm.notebook import tqdm
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchaudio
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as display
import librosa
import librosa.display
import pandas as pd
import seaborn as sns
import requests
import imageio
from zipfile import ZipFile
from PIL import Image, ImageOps, ImageShow
from torchvision.utils import save_image 

Pokemone sprites sourced from: https://pkmn.net/?action=content&page=viewpage&id=8644&parentsection=87
Includes sprites from BW, DPPt, FRLG, GSC, HGSS, Pinnball, RBYG, RSE. 

Utilizing AEGAN architechture from https://arxiv.org/abs/2004.05472,
code at: https://github.com/ConorLazarou/PokeGAN/blob/master/main.py

Note: Things changed:
- removed transparency from .png files, 

In [1]:
# Utils
def set_device():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device != "cuda":
        print("WARNING: For this notebook to perform best, "
          "if possible, in the menu under `Runtime` -> "
          "`Change runtime type.`  select `GPU` ")
    else:
        print("GPU is enabled in this notebook.")

    return device

def padding(img, expected_size):
    desired_size = expected_size
    delta_width = desired_size - img.size[0]
    delta_height = desired_size - img.size[1]
    pad_width = delta_width // 2
    pad_height = delta_height // 2
    
    new_img = Image.new(img.mode, (expected_size,expected_size), (255,255,255))
    new_img.paste(img, (pad_width, pad_height))
    #padding = (pad_width, pad_height, delta_width - pad_width, delta_height - pad_height)
    return new_img #ImageOps.expand(img, padding, fill = (255,255,255))


def resize_with_padding(img, expected_size):
    img.thumbnail((expected_size[0], expected_size[1]))
    delta_width = expected_size[0] - img.size[0]
    delta_height = expected_size[1] - img.size[1]
    pad_width = delta_width // 2
    pad_height = delta_height // 2
    padding = (pad_width, pad_height, delta_width - pad_width, delta_height - pad_height)
    return ImageOps.expand(img, padding)#,fill = 255)

def remove_transparency(im, bg_colour=(255, 255, 255)):

    # Only process if image has transparency (http://stackoverflow.com/a/1963146)
    if im.mode in ('RGBA', 'LA') or (im.mode == 'P' and 'transparency' in im.info):

        # Need to convert to RGBA if LA format due to a bug in PIL (http://stackoverflow.com/a/1963146)
        alpha = im.convert('RGBA').split()[-1]

        # Create a new background image of our matt color.
        # Must be RGBA because paste requires both images have the same format
        # (http://stackoverflow.com/a/8720632  and  http://stackoverflow.com/a/9459208)
        bg = Image.new("RGBA", im.size, bg_colour + (255,))
        bg.paste(im, mask=alpha)
        return bg

    else:
        return im

#def iter_frames(im):
#    try:
#        i= 0
#        while 1:
#            im.seek(i)
#            imframe = im.copy()
#            if i == 0: 
#                palette = imframe.getpalette()
#            else:
#                imframe.putpalette(palette)
#            yield imframe
#            i += 1
#    except EOFError:
#        pass
#
#    for i, frame in enumerate(iter_frames(im)):
#        frame.save('test%d.png' % i,**frame.info)   

In [7]:
src_path = "C:\\Users\\ipzac\\Documents\\Project Data\\Pokemon Sprites\\Original Pokémon\\"
save_path = "C:\\Users\\ipzac\\Documents\\Project Data\\Pokemon Sprites\\Clean Sprites\\"
os.chdir(src_path)
template = np.zeros((96,96))
src_files = []
unique_ID = 0

for root,d, files in os.walk(src_path):
    for f in files:
        im = Image.open(root + '\\' + f)
        im = remove_transparency(im)
        #bw_im = ImageOps.grayscale(im)
        pad_im = padding(im, 96)
        pad_im.save(save_path + str(unique_ID) + ".png")
        #tensor_im = transforms.ToTensor()(pad_im).squeeze()
        #torch.save(tensor_im, save_path + str(unique_ID) + ".pt")
        unique_ID += 1
        #ImageShow.show(pad_im)

In [4]:
set_device()

GPU is enabled in this notebook.


'cuda'

In [ ]:
file_path = "C:\\Users\\ipzac\\Documents\\Personal-Projects\\Sprite Image Generation\\"
os.chdir(file_path)

%run main.py

Epoch 1; Elapsed time = 00:00:00s
101/286: G=5.572, E=0.577, Dx=0.027, Dz=0.730  Rx=0.687  Rz=0.630
Epoch 2; Elapsed time = 00:01:12s
101/286: G=8.281, E=0.645, Dx=0.106, Dz=0.715  Rx=0.224  Rz=0.247
Epoch 3; Elapsed time = 00:02:22s
